In [1]:
import dill
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode()


/home/lucas/anaconda3/lib/python3.5/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
with open('/home/lucas/tesis2/test_outputs/sequence_tree_1000samples_20levels_100samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_1000 = dill.load(f)
    
with open('/home/lucas/tesis2/test_outputs/sequence_tree_5000samples_20levels_100samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_5000 = dill.load(f)
with open('/home/lucas/tesis2/test_outputs/sequence_tree_20000samples_20levels_100samples_per_class_20results_per_query.dill', 'rb') as f:
    test_results_20000 = dill.load(f)

In [3]:
def aggregate_results(results):
    aggregated_results = []
    for  class_, class_results in results.items():
        aggregated_results += list(class_results.values())
    return aggregated_results

def aggregate_results_fixed_n(model_results, n):
    aggregated_results = {}
    for class_name, results in model_results.items():
        nscores = [scores[n] for scores in results.values()]
        aggregated_results[class_name] = nscores
    return aggregated_results

In [15]:
def plot_scores(results, title):
    data = []
    for name, result in sorted(results.items()):
        x = []
        y = []
        for query in result:
            x += range(1, len(query) + 1)
            y += query.tolist()
        x = ['.{0}.'.format(i) for i in x]
        trace = go.Box(
            y=y,
            x=x,
            name=name,
            boxmean='sd'
        )
        data.append(trace)
        
    layout = go.Layout(
        yaxis=dict(
            title=title,
            zeroline=True
        ),
        boxmode='group'
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
    
def plot_classes(results_dict, title):
    data = []
    for model_name, result in sorted(results_dict.items()):
        x = []
        y = []
        for class_name, scores in sorted(result.items()):
            x += len(scores)*[class_name]
            y += scores
        trace = go.Box(
            y=y,
            x=x,
            name=model_name,
            boxmean=True
        )
        data.append(trace)
    layout = go.Layout(
        yaxis=dict(
            title=title,
            zeroline=True
        ),
        boxmode='group'
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [5]:
models = [test_results_1000, test_results_5000, test_results_20000]
models_results_by_class = [model['results'] for model in models]
models_names = ['1000', '5000', '20000']
models_aggregated_results = {name: aggregate_results(model) 
                             for name, model in zip(models_names, models_results_by_class)}
plot_scores(models_aggregated_results, 'nDCG')

In [6]:
model = models_results_by_class[2]
results = {class_name: list(results.values()) for class_name, results in model.items()}
plot_scores(results, 'nDCG')

In [8]:
model = models_results_by_class[0]
results = {class_name: list(results.values()) for class_name, results in model.items()}
plot_scores(results, 'nDCG')

In [22]:
n = 20
models_names = [int(name) for name in models_names]
results_by_class = {name: aggregate_results_fixed_n(model, n - 1)
                    for model, name in zip(models_results_by_class, models_names)}
plot_classes(results_by_class, 'nDCG')

In [23]:
results_by_class['1000']['BE']

KeyError: '1000'